# Projet de Machine Learning

In [1]:
from tools import *
import math as math
import copy
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def gen_arti(centerx=1,centery=1,sigma=0.1,nbex=10,data_type=0,eps=0.02):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param eps: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
         #melange de 2 gaussiennes
         xpos=np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//2)
         xneg=np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//2)
         data=np.vstack((xpos,xneg))
         y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,eps,nbex)
    data[:,1]+=np.random.normal(0,eps,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

    

In [114]:
def f_K(x, e, sigma=1):
    return math.exp(-pow(np.linalg.norm(x-e), 2)/(2*pow(sigma,2)))

def HA(a, S, dataset, y):
    N = dataset.shape[0]
    Kql= np.zeros((N,N))
    Kal = np.zeros((N,N))
    f = np.zeros((0, 0))
    
    print "S actuel:shape", len(S)
    print "_________"
    for el in S:
        print el
    print "*************\n\n"
    print "Calcul des fl..."
    for l,x in enumerate(dataset):
        fl = a[l]*f_K(x, x)
        for j,e in enumerate(S):
            fl += a[j]*K[l,j]
        f = np.append(f, fl)
        
    print "Done"

    for i,x in enumerate(S):
        for j,e in enumerate(S):
            Kql[i,j] = f_K(x,e)
            
    Kql = np.asmatrix(np.asarray(Kql))
    f = np.asmatrix(f.reshape((f.shape[0], 1)))
    y = np.asmatrix(y)

    
    print "Calcul de H(xl)...", -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a
    return -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a

def H(x, S, a_prec, dataset, y):

    a = a_optim(x, S, a_prec) 
    return HA(a, S, dataset, y)

def a_optim(x, S, a_prec):

    f = K*a_prec
    p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
    x = np.asarray(p*(1-p))
    W = np.asmatrix(np.zeros((x.shape[0], x.shape[0])))
    
    for i,element in enumerate(x):
        W[i,i] = element
        
    z = K*a_prec+np.linalg.inv(W)*(y-p)
    return np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 

def ivm(dataset, y):

    N = dataset.shape[0]
    a = np.zeros((N,1))
    R = np.zeros((N,2))
    S = []
    lamda = 3
    
    Hk = 1
    Hk_1 = 0
    
    for i, e in enumerate(dataset):
        R[i] = e

    while abs(Hk-Hk_1)/abs(Hk) > 0.01:
        h = np.zeros((N, 1))
        S_b = []
        for e in S:
            S_b.append(e)
        print "R shape:----->", R.shape
        for i, xl in enumerate(R):

            S_b.append(xl)
            print "-------------"
            print "S U xl", S_b
            print "-------------\n\n\n"
            print "Nombre d'elements dans la somme:", len(S_b)
            h[i] = H(xl, S_b, a, dataset, y)

        
        print "Done"
        print "On recupere le plus petit H(xl)"
        xl_opt = np.argmin(h)
        Hk_1 = Hk
        Hk = h[xl_opt]
        print "Le plus petit est ", Hk, " il est à l'indice : ", xl_opt

        print "On l'ajoute a S..."
        S.append(dataset[xl_opt])
        print "OK"
        print "On l'enleve a R... Taille avant:", R.shape
        data = np.zeros((R.shape[0]-1,2))
        for i, e in enumerate(R):
            if i == xl_opt:
                pass
            else:
                data[i] = e
        R = data
        print "R shape:", R.shape
        # update de a
        print "Updating a..."
        a_prec = a
        f = K*a_prec
        p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
        x = np.asarray(p*(1-p))
        W = np.asmatrix(np.zeros((x.shape[0], x.shape[0])))

        for i,element in enumerate(x):
            W[i,i] = element
        z = K*a_prec+np.linalg.inv(W)*(y-p)
        a = np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 
        print "OK"
        print "a shape:", a.shape
        print "ecart", abs(Hk-Hk_1)/abs(Hk)
        
    print "S", S


In [115]:
data, y = gen_arti()
data = np.asarray(data)
print data
N = data.shape[0]
K = np.asmatrix(np.zeros((N,N)))
lamda=1
for i,x in enumerate(data):
    for j,e in enumerate(data):
        K[i,j] = f_K(x,e)

y = y.reshape((y.shape[0], 1))
ivm(data, y)

[[ 0.44272317  0.80330648]
 [ 1.01763374  0.72529025]
 [-0.9106029  -0.65207142]
 [-1.28466633 -1.49898002]
 [-0.61910535 -1.05313228]
 [ 1.01602393  0.77420929]
 [ 1.09817334  1.23650283]
 [ 1.45652002  1.28527675]
 [-0.63573637 -0.92840275]
 [-0.6510425  -0.69522432]]
R shape:-----> (10, 2)
-------------
S U xl [array([ 0.44272317,  0.80330648])]
-------------



Nombre d'elements dans la somme: 1
S actuel:shape 1
_________
[ 0.44272317  0.80330648]
*************


Calcul des fl...
Done
Calcul de H(xl)... [[-54.04773558]]
-------------
S U xl [array([ 0.44272317,  0.80330648]), array([ 1.01763374,  0.72529025])]
-------------



Nombre d'elements dans la somme: 2
S actuel:shape 2
_________
[ 0.44272317  0.80330648]
[ 1.01763374  0.72529025]
*************


Calcul des fl...
Done
Calcul de H(xl)... [[-64.0094459]]
-------------
S U xl [array([ 0.44272317,  0.80330648]), array([ 1.01763374,  0.72529025]), array([-0.9106029 , -0.65207142])]
-------------



Nombre d'elements dans la somm